# Exercise 2: Nonholonomic Wheeled Robot Motion Simulation

The goal of this exercise is to build a Python class that defines the dynamics of a nonholonomic wheeled robot that we can model using a kinematic unicycle model.

Unlike in the `simulation.ipynb` notebook, here we will leverage Python classes and inheritance which are general programming fundamentals that we will use throughout the later exercises.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)

## Dynamics Base Class

We will first define a base `Dynamics` class that will be the core structure for future dynamics models, including the nonholonomic wheeled robot model. This base class is defined in `ch01/dynamics.py`.

In [ ]:
from dynamics import Dynamics
from IPython.display import Code
Code(filename='dynamics.py', language='python') 

## Nonholonomic Wheeled Robot Dynamics

We model the robot dynamics using the kinematic unicycle model:
\begin{equation}
\dot{x} = v \cos(\theta), \quad \dot{y} = v \sin(\theta), \quad \dot{\theta} = \omega.
\end{equation}
where $(x,y)$ is the position, $\theta$ is the heading angle with respect to the x-axis, and the control inputs are the speed $v$ and the angular velocity $\omega$.

In [ ]:
class RobotState():
  def __init__(self, x=0, y=0, θ=0):
    self.x  = x
    self.y  = y
    self.θ = θ

class RobotControl():
  def __init__(self, v=0, ω=0):
    self.v = v
    self.ω = ω

#### Exercise 2.1: Define Nonholonomic Wheeled Robot Dynamics Class

Below we define the `RobotDynamics` class by inheriting from the `Dynamics` base class. Complete the implementations of the `feed_forward` and `rollout` functions.

In [ ]:
class RobotDynamics(Dynamics):

  def __init__(self) -> None:
    super().__init__()
    self.n = 3  # length of state
    self.m = 2  # length of control

  def feed_forward(self, state:RobotState, control:RobotControl):
    # Define Gaussian Noise
    if self.noisy == False:
      var = np.array([0.0, 0.0, 0.0])
    elif self.noisy == True:
      var = np.array([0.01, 0.01, 0.001])
    w = np.random.normal(loc=np.array([0.0, 0.0, 0.0]), scale=var)


    state_new = RobotState()
    ############################## Code starts here ##############################
    """
    Populate "state_new" by applying discrete time dynamics equations. Use "self.dt" from the Dynamics base class.
    """

    ############################## Code ends here ##############################

    # Add noise
    state_new.x  = state_new.x  + w[0]
    state_new.y  = state_new.y  + w[1]
    state_new.th = state_new.th + w[2]
    return state_new

  def rollout(self, state_init, control_traj, num_rollouts):
    num_steps = control_traj.shape[1]

    state_traj_rollouts = np.zeros((self.n*num_rollouts, num_steps+1))
    ############################## Code starts here ##############################
    """
    Use two for-loops to loop through "num_rollouts" and "num_steps" to populate "state_traj_rollouts". Use the "feed_forward" function above.
    """

    ############################## Code ends here ##############################

    return state_traj_rollouts

Run the script below to simulate the robot motion and plot the state and control trajectories.

In [ ]:
# Run Robot Dynamics Rollouts

# Constants
num_rollouts = 10
num_steps = 100

# Define the dynamics class
tb_dynamics = RobotDynamics()

# Define the control inputs
control_traj = np.ones((tb_dynamics.m,num_steps))
control_traj[1,:] = np.sin(np.linspace(0.0, 2*np.pi, num_steps))

# Define inital state
state_init = RobotState(0.0, 0.0, 0.0)

# Rollouts dynamics
state_traj_rollouts = tb_dynamics.rollout(state_init, control_traj, num_rollouts)

# Plot control inputs
fig_ctrl, axs_ctrl = plt.subplots(nrows=tb_dynamics.m)
ylabels_ctrl = ["v", "ω"]
ylims_ctrl = np.array([[0.0, 2.0],[-1.1, 1.1]])
for j in range(tb_dynamics.m):
  axs_ctrl[j].plot(np.linspace(0,num_steps,num_steps), control_traj[j, :])
  axs_ctrl[j].set_ylabel(ylabels_ctrl[j])
  axs_ctrl[j].set_ylim(ylims_ctrl[j,:])
fig_ctrl.suptitle("Control")

# Plot state trajectory rollouts
fig_state, axs_state = plt.subplots(nrows=tb_dynamics.n)
ylabels_state = ["x", "y", "θ"]
ylims_state = np.array([[-0.1, 1.1], [-0.5, 0.5], [-0.1, 0.5]])
for i in range(num_rollouts):
  for j in range(tb_dynamics.n):
    axs_state[j].plot(np.linspace(0,num_steps+1,num_steps+1), state_traj_rollouts[tb_dynamics.n*i + j, :], c='c')
    axs_state[j].set_ylabel(ylabels_state[j])
    axs_state[j].set_ylim(ylims_state[j,:])
fig_state.suptitle("State")
plt.show()